To use this notebook in <a href="https://www.tutorialspoint.com/google_colab/google_colab_executing_external_python_files.htm">Google Colab</a>, please download the main folder from <a href="https://github.com/maiziezhoulab/RNN_BrainMaturation">here</a> and start from [Set up your Colab](#Set-up-your-Colab)

Or else, start from [Train&Analysis](#Train&Analysis)

# Set up your Colab
## Install Tensorflow

In [ ]:
!/opt/bin/nvidia-smi
!pip install tensorflow-gpu==1.15.0

## Mount your Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

## Change dir to the script

In [ ]:
import os
path = "/content/drive/My Drive/Colab_test"#example:"/content/drive/My Drive/RNN_BrainMaturation-master"
os.chdir(path)
!pwd

# Train&Analysis
## GPU set up

In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## Train
### Set parameters

In [ ]:
model_dir = 'data/6tasks'

#Hyper parameters
hp = {'activation': 'softplus',
    'n_rnn': 256,
    'learning_rate': 0.001,
    'mix_rule': True,
    'l1_h': 0.,
    'use_separate_input': False,
    'target_perf': 0.995,
    'mature_target_perf': 0.95,
    'mid_target_perf': 0.65,
    'early_target_perf': 0.35,}

# rule set and rule trains
ruleset='all_new'
rule_trains=['overlap','zero_gap','gap','odr','odrd','gap500',]

### Start Training

In [ ]:
from training.train import train

train(model_dir,
      seed=0,
      hp=hp,
      ruleset=ruleset,
      rule_trains=rule_trains,
      display_step=20)

## Analysis
### Set model dir

In [7]:
model_dir = 'data/6tasks'

### Load Hyper parameter and training log

In [8]:
from utils import tools
hp = tools.load_hp(model_dir)
log = tools.load_log(model_dir)

### Print basic information

In [ ]:
from analysis.PSTH_print_basic_info import print_basic_info
trial_selected = print_basic_info(hp,log,model_dir,smooth_growth=True,smooth_window=5,\
                                  avr_window=9,perf_margin=0.05,max_trial_num_limit=50,auto_range_select=True)

### Compute H
Run this block if you need to save up storage

In [ ]:
from analysis.PSTH_compute_H import gen_task_info
gen_task_info(hp,log,model_dir,['odr','odrd','overlap','zero_gap','gap']) #only generate task_info.pkl

Run this block if you have enough storage and want to speed up the analysis process

In [ ]:
#recompute = False
#from analysis.PSTH_compute_H import compute_H
#compute_H(hp,log,model_dir,rules=['odr','odrd','overlap','zero_gap','gap'],\
#          trial_list=trial_selected,recompute=recompute,)

### Generate neuron information

In [ ]:
recompute = False
from analysis.PSTH_gen_neuron_info import generate_neuron_info
generate_neuron_info(hp,model_dir,epochs=['stim1','delay1'],trial_list=trial_selected,rules=['odr','odrd'],recompute=recompute)
generate_neuron_info(hp,model_dir,epochs=['delay2'],trial_list=trial_selected,rules=['odrd'],recompute=recompute)
generate_neuron_info(hp,model_dir,epochs=['stim1'],trial_list=trial_selected,rules=['overlap','zero_gap','gap'],recompute=recompute)

### Plot tuning curve

In [ ]:
from analysis.PSTH_tunning_analysis import tunning_analysis
tunning_analysis(hp,log,model_dir,'odr','stim1',trial_selected)
tunning_analysis(hp,log,model_dir,'odr','delay1',trial_selected)
tunning_analysis(hp,log,model_dir,'odrd','stim1',trial_selected)
tunning_analysis(hp,log,model_dir,'odrd','delay2',trial_selected)
tunning_analysis(hp,log,model_dir,'gap','stim1',trial_selected)
tunning_analysis(hp,log,model_dir,'overlap','stim1',trial_selected)
tunning_analysis(hp,log,model_dir,'zero_gap','stim1',trial_selected)

### Plot PSTH

In [ ]:
from analysis.PSTH_plot_PSTH import plot_PSTH
plot_PSTH(hp, log, model_dir, 'odr', 'stim1', trial_selected, plot_oppo_dir = False)
plot_PSTH(hp, log, model_dir, 'odr', 'delay1', trial_selected, plot_oppo_dir = False)
plot_PSTH(hp, log, model_dir, 'odrd', 'stim1', trial_selected, plot_oppo_dir = True)
plot_PSTH(hp, log, model_dir, 'odrd', 'delay1', trial_selected, plot_oppo_dir = True)
plot_PSTH(hp, log, model_dir, 'odrd', 'delay2', trial_selected, plot_oppo_dir = True)

plot_PSTH(hp, log, model_dir, 'gap', 'stim1', trial_selected, plot_oppo_dir = False)
plot_PSTH(hp, log, model_dir, 'zero_gap', 'stim1', trial_selected, plot_oppo_dir = False)
plot_PSTH(hp, log, model_dir, 'overlap', 'stim1', trial_selected, plot_oppo_dir = False)

### Plot the growth of the epoch mean value

In [ ]:
from analysis.PSTH_gen_PSTH_log import gen_PSTH_log
from analysis.PSTH_plot_epoch_mean_growth import plot_epoch_mean_growth

PSTH_log = gen_PSTH_log(hp,log['trials'],model_dir,rule = 'odr',seltive_epoch = 'stim1')
plot_epoch_mean_growth(hp,log, log['trials'], model_dir, rule = 'odr', seltive_epoch = 'stim1', analy_epoch = 'stim1',PSTH_log = PSTH_log)
#plot_epoch_mean_growth(hp,log, log['trials'], model_dir, rule = 'odr', seltive_epoch = 'stim1', analy_epoch = 'delay1',PSTH_log = PSTH_log)

#plot_epoch_mean_growth(hp,log, log['trials'], model_dir, rule = 'odr', seltive_epoch = 'delay1', analy_epoch = 'delay1',)

#PSTH_log = gen_PSTH_log(hp,log['trials'],model_dir,rule = 'odrd',seltive_epoch = 'stim1')
#plot_epoch_mean_growth(hp,log, log['trials'], model_dir, rule = 'odrd', seltive_epoch = 'stim1', analy_epoch = 'stim1',PSTH_log = PSTH_log)
#plot_epoch_mean_growth(hp,log, log['trials'], model_dir, rule = 'odrd', seltive_epoch = 'stim1', analy_epoch = 'delay2',PSTH_log = PSTH_log)

### Plot sample neurons

In [15]:
from analysis.PSTH_sample_neuron_by_trial import sample_neuron_by_trial
sample_neuron_by_trial(hp,log,model_dir,'odr','stim1',trial_selected['odr']['mature'][-1:],'mix_neurons')
#sample_neuron_by_trial(hp,log,model_dir,'odr','delay1',[616960,628480],'mix_neurons')
sample_neuron_by_trial(hp,log,model_dir,'odrd','stim1',trial_selected['odrd']['mature'][-1:],'mix_neurons')
#sample_neuron_by_trial(hp,log,model_dir,'odrd','delay2',[616960,628480],'mix_neurons')

#sample_neuron_by_trial(hp,log,model_dir,'odr','stim1',[616960,628480],'exh_neurons')
#sample_neuron_by_trial(hp,log,model_dir,'odr','delay1',[616960,628480],'exh_neurons')
#sample_neuron_by_trial(hp,log,model_dir,'odrd','stim1',[616960,628480],'exh_neurons')
#sample_neuron_by_trial(hp,log,model_dir,'odrd','delay2',[616960,628480],'exh_neurons')

#Choose trial for Decoder and dPCA analysis

In [16]:
import numpy as np
chosen_trial = dict()
for rule in trial_selected.keys():
  chosen_trial[rule]=dict()
  for mature,sel_trails in trial_selected[rule].items():
    pref_diff_list = [abs(log['perf_'+rule][int(i/log['trials'][1])]-hp[mature+'_target_perf']) for i in sel_trails]
    chosen_trial[rule][mature] = sel_trails[pref_diff_list.index(min(pref_diff_list))]

### Decoder
#### ODR

In [17]:
from analysis.PSTH_Decoder_analysis import Decoder_analysis
Decoder_analysis(hp,model_dir,trial_num = chosen_trial['odr']['mature'],window_size=10,test_num=2)
Decoder_analysis(hp,model_dir,trial_num = chosen_trial['odr']['mid'],window_size=10,test_num=2)
Decoder_analysis(hp,model_dir,trial_num = chosen_trial['odr']['early'],window_size=10,test_num=2)

#### ODRD

In [18]:
import numpy as np
from analysis.PSTH_Decoder_analysis import Decoder_analysis
Decoder_analysis(hp,model_dir,trial_num = chosen_trial['odrd']['mature'],window_size=10,test_num=2,rule='odrd',)
Decoder_analysis(hp,model_dir,trial_num = chosen_trial['odrd']['mid'],window_size=10,test_num=2,rule='odrd',)
Decoder_analysis(hp,model_dir,trial_num = chosen_trial['odrd']['early'],window_size=10,test_num=2,rule='odrd',)

### Demixed PCA
#### Install dPCA

In [ ]:
!pip install dpca

#### ODR

In [ ]:
from analysis.PSTH_neuron_dPCA import neuron_dPCA
neuron_dPCA(hp,log,model_dir,trial_num = chosen_trial['odr']['mature'],rule='odr',task_mode='test',invert_tcomp_y=True,)
neuron_dPCA(hp,log,model_dir,trial_num = chosen_trial['odr']['mid'],rule='odr',task_mode='test',invert_tcomp_y=True,)
neuron_dPCA(hp,log,model_dir,trial_num = chosen_trial['odr']['early'],rule='odr',task_mode='test',invert_tcomp_y=True,)

#### ODRD

In [ ]:
from analysis.PSTH_neuron_dPCA import neuron_dPCA
neuron_dPCA(hp,log,model_dir,trial_num = chosen_trial['odrd']['mature'],rule='odrd',task_mode='test',appoint_loc_analysis=True, appoint_locs=[1,5],\
            invert_tcomp_y=True,invert_stcomp_y=True,tcomp_ylim=(-12,10),scomp_ylim=(-1,1),stcomp_ylim=(-15,15))
neuron_dPCA(hp,log,model_dir,trial_num = chosen_trial['odrd']['mid'],rule='odrd',task_mode='test',appoint_loc_analysis=True, appoint_locs=[1,5],\
            invert_tcomp_y=True,invert_stcomp_y=True,tcomp_ylim=(-12,10),scomp_ylim=(-1,1),stcomp_ylim=(-15,15))
neuron_dPCA(hp,log,model_dir,trial_num = chosen_trial['odrd']['early'],rule='odrd',task_mode='test',appoint_loc_analysis=True, appoint_locs=[1,5],\
            invert_tcomp_y=True,invert_stcomp_y=True,tcomp_ylim=(-12,10),scomp_ylim=(-1,1),stcomp_ylim=(-15,15))

#### Overlap

In [ ]:
from analysis.PSTH_neuron_dPCA import neuron_dPCA
neuron_dPCA(hp,log,model_dir,trial_num = chosen_trial['overlap']['mature'],rule='overlap',task_mode='test',appoint_loc_analysis=True, appoint_locs=[0,4],\
            invert_scomp_y=True,invert_stcomp_y=True,tcomp_ylim=(-4,8),scomp_ylim=(-1.5,1.5),stcomp_ylim=(-3,3))
neuron_dPCA(hp,log,model_dir,trial_num = chosen_trial['overlap']['mid'],rule='overlap',task_mode='test',appoint_loc_analysis=True, appoint_locs=[1,5],\
            invert_scomp_y=True,invert_stcomp_y=True,tcomp_ylim=(-4,8),scomp_ylim=(-1.5,1.5),stcomp_ylim=(-3,3))
neuron_dPCA(hp,log,model_dir,trial_num = chosen_trial['overlap']['early']
            ,rule='overlap',task_mode='test',appoint_loc_analysis=True, appoint_locs=[1,5],\
            invert_scomp_y=True,invert_stcomp_y=True,tcomp_ylim=(-4,8),scomp_ylim=(-1.5,1.5),stcomp_ylim=(-3,3))